<a id="topC"></a>

# Running the COS Pipeline (`calcos`)

# Learning Goals
### This Notebook is designed to walk the user (*you*) through:
#### 1. **[Setting up the environment to run `calcos`](#setupC)**
##### $\ \ \ \ $ 1.1. [Prerequisites](#prereqC)
##### $\ \ \ \ $ 1.2. [Create your conda environment](#condaenvC)


#### 2. **[Processing raw COS data using `calcos`](#runC)**
#### 3. **[Re-processing data COS data with altered parameters](#rerunC)**

# 0. Introduction
#### The Cosmic Origins Spectrograph ([*COS*](https://www.nasa.gov/content/hubble-space-telescope-cosmic-origins-spectrograph)) is an ultraviolet spectrograph on-board the Hubble Space Telescope ([*HST*](https://www.stsci.edu/hst/about)) with capabilities in the near ultraviolet (*NUV*) and far ultraviolet (*FUV*).

#### This tutorial aims to prepare you to begin analyzing COS data of your choice by walking you through reading and viewing a spectrum obtained with COS, as well as obtaining a few diagnostic measurements of your spectrum.

- For an in-depth manual to working with COS data and a discussion of caveats and user tips, see the [COS Data Handbook](https://hst-docs.stsci.edu/display/COSDHB/).
- For a detailed overview of the COS instrument, see the [COS Instrument Handbook](https://hst-docs.stsci.edu/display/COSIHB/).



#### **`Calcos`** is the data processing pipeline which converts the raw data produced by COS's detectors onboard Hubble into usable spectral data. It transitions the data from a list of many individual recorded photon interactions into tables of wavelength and flux.

<a id = setupC></a>
# 1. Setting up the environment to run calcos

The first step to processing your data is setting up an environment from which to run `calcos`.
<a id = prereqC></a>
## 1.1. Prerequisites
This tutorial assumes some basic knowledge of the command line and was built using a unix bash-style shell. Those using a Windows computer will likely have the best results if working within the [Windows Subsystem for Linux](https://docs.microsoft.com/en-us/windows/wsl/install-win10).


If you do not already have any distribution of the `conda` tool, see [this page](https://astroconda.readthedocs.io/en/latest/getting_started.html#getting-started-jump) for instructions, and install either [`anaconda` - more beginner friendly, \~ 3 GB, lots of extras you likely won't use](https://docs.anaconda.com/anaconda/install/) or [`miniconda` - \~ 400 MB, only what you need](https://docs.conda.io/en/latest/miniconda.html).

<a id = condaenvC></a>
## 1.2. Create your conda environment
Once you have `conda` installed, it's time to create an environment. 

Open up your terminal app, likely `Terminal` or `iTerm` on a Mac or `Windows Terminal` or `Powershell` on Windows.

First, add the stsci channel to your computer's conda channel list. This enables conda to look in the right place to find all the packages we want to install.


``` $ conda config --add channels http://ssb.stsci.edu/astroconda```


Now we can create a new environment for running calcos; let's call it `calcos_env`, and initialize it with the packages in the stsci channel's list we just added.


``` $ conda create -n astroconda stsci ```

After allowing conda to proceed to installing the packages (type `y` then hit enter/return), you can see all of your environments with:

``` $ conda env list```

and then switch over to your new environment with 

``` $ conda activate calcos_env ```

At this point, typing `calcos` into the command line and hitting enter should no longer yield the error 

> ```command not found: calcos``` 

but rather respond that:

> ```The command-line options are:
  --version (print the version number and exit)
  -r (print the full version string and exit)
  ...
  ERROR:  An association file name or observation rootname must be specified.```

<a id = runC></a>
# 2. Processing raw COS data using `calcos`

<a id = rerunC></a>
# 3. Re-processing data COS data with altered parameters